# Agent 2: Cyber Collector

In this multi-agent system, **`Cyber Collector`** serves as the first agent for information collection and organization, with its main task being to provide high-quality, structured information input for subsequent agents.

In [10]:
# load environment variables from the .env file
from dotenv import load_dotenv

load_dotenv()

True

In [11]:
import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential


project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(), conn_str=os.environ["AIPROJECT_CONNECTION_STRING"]
)

## Define knowledge base

In [12]:
# Upload file and create vector store
file = project_client.agents.upload_file_and_poll(
    file_path="./data/cyber_security_framework.md", purpose="assistants")
print(f"Uploaded file, file ID: {file.id}")

vector_store = project_client.agents.create_vector_store_and_poll(
    file_ids=[file.id], name="my_vectorstore")
print(f"Created vector store, vector store ID: {vector_store.id}")

Uploaded file, file ID: assistant-AjfWW2aiDbjWkHGCnxwZ2q
Created vector store, vector store ID: vs_Ole0P3cnMhD8llgF1JNo9ZEs


## Create Agent

In [13]:
from azure.ai.projects.models import (
    FileSearchTool,
)

# Create file search tool with resources followed by creating agent
file_search = FileSearchTool(vector_store_ids=[vector_store.id])

cyber_collector = project_client.agents.create_agent(
    model=os.environ["CHAT_MODEL"],
    name="cyber_collector",
    description="The cyber collector can collect information about cyber security from its knowledge base.",
    instructions="Hello, you are helpful assistant and can search information from uploaded files.",
    tools=file_search.definitions,
    tool_resources=file_search.resources,
    # Parameters
    temperature=0.7,
    top_p=0.95,
    # Metadata
    metadata={"group": "internet_threat_analysis"},
)

print(f"Created agent, agent ID: {cyber_collector.id}")

Created agent, agent ID: asst_fBZqcSLGQdT9StdR0OPTAxLk


In [14]:
agent_id = cyber_collector.id

thread = project_client.agents.create_thread()
print(f"Created thread, ID: {thread.id}")

# Create message to thread
message = project_client.agents.create_message(
    thread_id=thread.id, role="user", content="Retrieve the latest threat information for cybersecurity"
)
print(f"Created message, ID: {message.id}")

# Create and process assistant run in thread with tools
run = project_client.agents.create_and_process_run(
    thread_id=thread.id, assistant_id=agent_id)
print(f"Run finished with status: {run.status}")

if run.status == "failed":
    print(f"Run failed: {run.last_error}")

Created thread, ID: thread_nI8vk1kNTYAHSLUpDMZByWVG
Created message, ID: msg_w0MjHlgBHFLeMm1JmYZan7h9
Run finished with status: RunStatus.COMPLETED


In [15]:
# Display all messages in the thread
from IPython.display import Markdown, display
import helper

messages = project_client.agents.list_messages(thread_id=thread.id)

display(Markdown(helper.get_conversation_md(messages)))

# Conversation
___
### **User** (2025-04-02 11:00:44 台北標準時間)
Retrieve the latest threat information for cybersecurity
___
### **Assistant** (2025-04-02 11:00:48 台北標準時間)
The latest threat information for cybersecurity includes various Key Performance Indicators (KPIs) that organizations are monitoring to assess their security posture. Key metrics include:

1. **Intrusion Attempts**: Tracking unauthorized access attempts to identify patterns and vulnerabilities.
2. **Incident Detection Rate**: Measuring the percentage of security threats successfully identified by monitoring systems.
3. **Mean Time to Detect/Mean Time to Respond (MTTD/MTTR)**: Evaluating how quickly incidents are detected and responded to.
4. **Data Breach Volume**: Assessing the quantity and sensitivity of compromised information.
5. **Patch Compliance**: Monitoring the implementation of security patches across the infrastructure.
6. **Vulnerability Management**: Tracking identified vulnerabilities and their remediation progress.

These metrics are essential for organizations to strengthen their security measures and enhance their defensive capabilities【4:0†source】.
___

In [16]:
# Print citations from the messages
for citation in messages.file_citation_annotations:
    print(citation)

{'type': 'file_citation', 'text': '【4:0†source】', 'start_index': 967, 'end_index': 979, 'file_citation': {'file_id': 'assistant-AjfWW2aiDbjWkHGCnxwZ2q'}}


In [18]:
# Get messages from the thread
messages = project_client.agents.list_messages(thread_id=thread.id)
message_data = project_client.agents.list_messages(thread_id=thread.id).data
for msg in message_data:
    print(msg.content)

[{'type': 'text', 'text': {'value': 'The latest threat information for cybersecurity includes various Key Performance Indicators (KPIs) that organizations are monitoring to assess their security posture. Key metrics include:\n\n1. **Intrusion Attempts**: Tracking unauthorized access attempts to identify patterns and vulnerabilities.\n2. **Incident Detection Rate**: Measuring the percentage of security threats successfully identified by monitoring systems.\n3. **Mean Time to Detect/Mean Time to Respond (MTTD/MTTR)**: Evaluating how quickly incidents are detected and responded to.\n4. **Data Breach Volume**: Assessing the quantity and sensitivity of compromised information.\n5. **Patch Compliance**: Monitoring the implementation of security patches across the infrastructure.\n6. **Vulnerability Management**: Tracking identified vulnerabilities and their remediation progress.\n\nThese metrics are essential for organizations to strengthen their security measures and enhance their defensive

In [9]:
# Delete the file when done
project_client.agents.delete_vector_store(vector_store.id)
print("Deleted vector store")

project_client.agents.delete_file(file_id=file.id)
print("Deleted file")

# Delete the agent when done
project_client.agents.delete_agent(agent_id)
print("Deleted agent")

Deleted vector store
Deleted file
Deleted agent
